# Upper and lower convex hull
### method of computing the convex hull of a finite set of points in 2-dimensional space


In [1]:
from ipynb.fs.full.drawing_tool import *
%matplotlib notebook
Tolerance = 10e-12

## 1. Auxiliary function:

In [ ]:
def orient(a, b, c):
    
    det_ = a[0] * b[1] + b[0] * c[1] + c[0] * a[1] - c[0] * b[1] - b[0] * a[1] - a[0] * c[1]
    
    if det_ > Tolerance:
        return 1
    elif det_ < -Tolerance:
        return -1
    return 0

## 2. Main algorithm:

In [ ]:
def make_convex_hull(data):
    def add_scene(flag):
#         wizualizacja
        nonlocal down_hull, up_hull, data, scenes
        down_lines = []
        up_lines = []
        check_line = []
        check_point = []
        if len(down_hull) >= 2:
            size_down = len(down_hull) - 1
            if flag == 'down':
                size_down -= 1
                check_line.append([down_hull[-2], down_hull[-1]])
                check_point.append(down_hull[-1])
            down_lines = [(down_hull[i], down_hull[i + 1]) for i in range(size_down)]
        if len(up_hull) >= 2:
            size_up = len(up_hull) - 1
            if flag == 'up':
                size_up -= 1
                check_line.append([up_hull[-2], up_hull[-1]])
                check_point.append(up_hull[-1])
            up_lines = [(up_hull[i], up_hull[i + 1]) for i in range(size_up)]
        scenes.append(Scene([PointsCollection(check_point, color='deeppink'),
                             PointsCollection(data, color='skyblue'),
                             PointsCollection(down_hull.copy(), color='limegreen'),
                             PointsCollection(up_hull.copy(), color='pink')],
                            [LinesCollection(down_lines, color='limegreen'),
                             LinesCollection(up_lines, color='pink'),
                            LinesCollection(check_line, color='deeppink')]))
    scenes = []
    points = sorted(data, key = lambda p: (p[0], p[1]))
    size = len(points)
#     pierwsze punkty otoczki:
    xmin_ymin = points[0]
    xmin_ymax = points[0]
    i = 1
    down_hull = []
    up_hull = []
    while xmin_ymax[0] == points[i][0]:
        xmin_ymax = points[i]
        i += 1
    first_ind = i - 1
    down_hull.append(xmin_ymin)
    scenes.append(Scene([PointsCollection(data, color='skyblue'),
                     PointsCollection(down_hull.copy(), color='limegreen')]))
#     znajdowanie dolnej otoczki
    for i in range(1, size):
        down_hull.append(points[i])
        while len(down_hull) > 2 and orient(down_hull[-3], down_hull[-2], down_hull[-1]) != 1:
            add_scene('down')
#             usuwamy przedostatni dopoki 3 ostatn
            down_hull.pop(-2)
        add_scene('none')
    
    up_hull.append(xmin_ymax)
    for i in range(first_ind + 1, size):
        up_hull.append(points[i])
        while len(up_hull) > 2 and orient(up_hull[-3], up_hull[-2], up_hull[-1]) != -1:
            add_scene('up')
            up_hull.pop(-2)
        add_scene('none')
    
    if up_hull[-1] == down_hull[-1]:
        up_hull.pop()
    for i in range(len(up_hull) - 1, 0, -1):
        down_hull.append(up_hull[i])
    if up_hull[0] != down_hull[0]:
        down_hull.append(up_hull[0])
    
    lines = []
    if len(down_hull) >= 2:
        lines = [(down_hull[i], down_hull[(i + 1) % len(down_hull)]) for i in range(len(down_hull))]
    scenes.append(Scene([PointsCollection(data, color='skyblue'),
                         PointsCollection(down_hull.copy(), color='royalblue')],
                        [LinesCollection(lines, color='royalblue')]))
    return scenes

##### Loading a set of points from a json file:

In [ ]:
with open('points.json', 'r') as file:
    data = js.loads(file.read())

In [ ]:
scenes = make_convex_hull(data)
plot = Plot(scenes = scenes)
plot.draw()

## 3. Points generators:

Returns set of points with vertexes of the rectangle and randomly placed points on the sides and diagonals of this rectangle:

In [ ]:
def on_retangle(n_sides, n_diagonals, vertexes):

    points = [vertexes[0], vertexes[1], vertexes[2], vertexes[3]]
    
    for i in range(n_sides):
        x1 = random.uniform(vertexes[0][0], vertexes[1][0])
        y1 = random.uniform(vertexes[0][1], vertexes[1][1])
        points.append((x1, y1))
        x2 = random.uniform(vertexes[0][0], vertexes[3][0])
        y2 = random.uniform(vertexes[0][1], vertexes[3][1])
        points.append((x2, y2))
        
    for i in range(n_diagonals):
        x1 = random.uniform(vertexes[0][0], vertexes[1][0])
        y1 = x1 + vertexes[0][1]
        points.append((x1, y1))
        x2 = random.uniform(vertexes[3][0], vertexes[2][0])
        y2 = -x2 + vertexes[3][1]
        points.append((x2, y2))
        
    return points

Returns set of points randomly placed on the circle:

In [ ]:
def on_circle(n, s, r):

    points = []
    for i in range(n):
        a = random.uniform(0, 2*np.pi)
        x = np.cos(a) * (r ** 2) + s[0]
        y = np.sin(a) * (r ** 2) + s[1]
        points.append((x, y))
        
    return points

Returns set of points randomly placed inside the rectangle:

In [ ]:
def randoms(n, p_x, p_y):

    points = []
    for _ in range(n):
        x = random.uniform(p_x[0], p_x[1])
        y = random.uniform(p_y[0], p_y[1])
        points.append((x, y))
        
    return points

## 4. For saving points entered with the mouse:

In [ ]:
def save_plot(plot, name):
    points = []
    for i in range(len(plot.get_added_points())):
        for point in plot.get_added_points()[i].points:
            points.append(point)

    with open(f'{name}.json', 'w') as file:
       file.write(js.dumps(points))

In [ ]:
plot = Plot(scenes=[Scene()])
plot.draw()

In [ ]:
save_plot(plot, "ccc")

## 5. Examples:

In [ ]:
data = randoms(100, [0, 100], [0, 100])
scenes = make_convex_hull(data)
plot = Plot(scenes=scenes)
plot.draw()

In [ ]:
data = on_retangle(50, 50, [(0, 0), (1, 0), (1, 1), (0, 1)])
scenes = make_convex_hull(data)
plot = Plot(scenes=scenes)
plot.draw()

In [ ]:
data = on_circle(100, (0, 0), 1)
scenes = make_(data)
plot = Plot(scenes=scenes)
plot.draw()